In [53]:
# Load the "autoreload" extension. Prior to executing code, modules are reloaded. 
# There's no need to restart jupyter notebook if you modify code in the `src` directory.
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset
import pandas as pd
from langdetect import detect
import string
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import requests
import boto3
import json
import pickle

import nltk
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

pd.set_option('display.max_rows',500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adambarnhard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adambarnhard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
########################################################
### Import Dataset
########################################################

In [44]:
##TODO: how should files be referenced properly?
raw_github_data = pd.read_csv('../data/raw/2020-04-06.tsv', sep='\t', header=0)


In [45]:
########################################################
### Clean Dataset
########################################################

In [46]:
# Filtering down to repos that are likely needing contributors based on past behavior
raw_github_data_filtered = raw_github_data[(raw_github_data['has_merged_prs'] == True) &
    (raw_github_data['has_readme'] == True) &
    (pd.isna(raw_github_data['repo_description']) == False) &
    (pd.isna(raw_github_data['primary_language_name']) == False) &
    (raw_github_data['count_distinct_contributors'] >=2)
]

In [47]:
# Detect language with error handling
def detect_with_error_handle(x):
    try:
        return detect(x)
    except:
        return 'Error'
    
# Check for only latin characters
def has_only_latin_letters(text):
    char_set = string.printable + '—'
    return all((True if x in char_set else False for x in text))

# Remove punctuation
def remove_punctuation(text):
    punctuation_list = string.punctuation + '—'
    return text.translate(str.maketrans('', '', punctuation_list))

In [48]:
## Full set of text processing

# check language, limit to english, and limit repo's with latin characters. Emojis are converted in the process
raw_github_data_filtered['language'] = raw_github_data_filtered['repo_description'].apply(lambda x: 'None' if pd.isna(x) else detect_with_error_handle(str(x)))
raw_github_data_filtered = raw_github_data_filtered[raw_github_data_filtered['language'] == 'en'].copy()
raw_github_data_filtered['is_latin_only_characters'] = raw_github_data_filtered['repo_description'].apply(lambda x: has_only_latin_letters(emoji.demojize(x)))
raw_github_data_filtered = raw_github_data_filtered[raw_github_data_filtered['is_latin_only_characters'] == True].copy()

# clean up repo description, topic, and language, combine into one big bag o' words
raw_github_data_filtered['repo_description_cleaned'] = raw_github_data_filtered['repo_description'].apply(lambda x: remove_punctuation(x))
raw_github_data_filtered['topics'] = raw_github_data_filtered.apply(lambda x: remove_punctuation(str(x['topics']).replace(',','').replace('nan','')), axis=1)
raw_github_data_filtered['topics'].fillna('', inplace=True)
raw_github_data_filtered['description_plus_topics'] = raw_github_data_filtered['repo_description_cleaned']+' '+raw_github_data_filtered['topics']+' '+raw_github_data_filtered['primary_language_name']
raw_github_data_filtered.reset_index(drop=True, inplace=True)

# create repo-lookup object for later use
repo_lookup = raw_github_data_filtered.copy()

<ipython-input-48-a0a382101f8b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_github_data_filtered['language'] = raw_github_data_filtered['repo_description'].apply(lambda x: 'None' if pd.isna(x) else detect_with_error_handle(str(x)))


In [49]:
########################################################
### Tokenize
########################################################

In [50]:
# Create class to be used by tokenizer to lemmatize... which change matches words to their roots
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [51]:
# Create a list of stop words that should be removed before tokenizing
stopwords = list(ENGLISH_STOP_WORDS) + ['covid19','coronavirus','virus','corona','covid','pandemic','sarscov2','outbreak','19','disease','2019','2019ncov','cord19','repository','repo','2020','20','covid2019','covidvirus', 'cases','case']

# Create vectorizor of n-grams using stop words and lemmatizer
word_vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word',stop_words=stopwords, tokenizer=LemmaTokenizer())

# Fit vectorizer on existing list of repos and create sparse matrix
sparse_vector_matrix = word_vectorizer.fit_transform(raw_github_data_filtered['description_plus_topics'])

In [52]:
########################################################
### Export Objects
########################################################

In [55]:
with open('../models/word_vectorizer.pickle', 'wb') as f:
    pickle.dump(word_vectorizer, f)
    
with open('../models/sparse_vector_matrix.pickle', 'wb') as f:
    pickle.dump(sparse_vector_matrix, f)

with open('../models/repo_lookup.pickle', 'wb') as f:
    pickle.dump(repo_lookup, f)